In [1]:
import sys

sys.path.append('D:\PythonProjects\\bk-monitor\\venv2\share\scripts')
import hello

当前运行环境缺少部分需要的环境变量，可能导致项目运行报错，请确认以下环境变量已经配置并赋值: BK_PAAS2_URL
注意，当前版本为社区版环境，请确认 BKPAAS_MAJOR_VERSION 已经配置并正确赋值


In [3]:
from core.drf_resource import resource
from bkmonitor.models.base import Shield
from mock import MagicMock
import copy

new_shield = {
    "category": "scope",
    "begin_time": "2024-12-10 00:00:00",
    "end_time": "2024-12-19 23:59:59",
    "cycle_config": {
        "begin_time": "",
        "end_time": "",
        "day_list": [],
        "week_list": [],
        "type": 1
    },
    "shield_notice": False,
    "notice_config": {},
    "description": "label 标签第三次测试",
    "dimension_config": {
        "scope_type": "instance",
        "target": [
            69
        ]
    },
    "bk_biz_id": 2
}


def create_shield(label):
    Shield.objects.all().delete()
    shield = copy.deepcopy(new_shield)
    shield["label"] = label
    resource.shield.add_shield(shield)
    model = Shield.objects.get(label=label)
    return model

In [3]:
# 测试添加屏蔽
def test_add_shield_resource():
    Shield.objects.all().delete()
    shield = copy.deepcopy(new_shield)

    label = "测试test_add_shield_resource"
    shield["label"] = label
    resource.shield.add_shield(shield)

    model = Shield.objects.get(label=label)
    assert model.label == label


test_add_shield_resource()

In [4]:
new_shield_2 = {
    "bk_biz_id": 2,
    "category": "alert",
    "begin_time": "2024-09-03 17:31:48",
    "end_time": "2024-09-03 18:01:48",
    "dimension_config": {
        "alert_ids": [
            "111", "100"
        ],
        # 编辑了维度后， 则将告警对应的剩下的维度的key放进来。 作为字典， key是告警id， value是剩下的维度key列表
        "dimensions": {"111": ["xxx", "yyy"], "100": ["xxx", "yyy"]},
    },
    "shield_notice": False,
    "description": "test",
    "cycle_config": {
        "begin_time": "",
        "type": 1,
        "day_list": [],
        "week_list": [],
        "end_time": ""
    }
}


# 测试批量添加屏蔽
def test_bulk_add_shield_resource():
    label = "测试test_bulk_add_shield_resource"
    new_shield_2["label"] = label
    instance = resource.shield.bulk_add_alert_shield
    instance.handle_alerts = MagicMock(return_value=[{"111": ["xxx", "yyy"], "100": ["xxx", "yyy"]}])
    # 创建告警屏蔽
    instance.request(new_shield_2)
    model = Shield.objects.get(label=label)
    assert model.label == label


test_bulk_add_shield_resource()

In [4]:
# 测试修改屏蔽
def test_edit_shield_resource():
    Shield.objects.all().delete()
    label = "测试test_edit_shield_resource"

    shield = copy.deepcopy(new_shield)

    shield["label"] = label
    resource.shield.add_shield(shield)
    model = Shield.objects.get(label=label)
    print(model.label, model.id)

    shield["label"] = "测试test_edit_shield_resource_2"
    shield["id"] = model.id
    resource.shield.edit_shield(shield)
    model = Shield.objects.get(id=model.id)
    print(model.label)
    assert model.label == "测试test_edit_shield_resource_2"

    del shield["label"]
    resource.shield.edit_shield(shield)
    model = Shield.objects.get(id=model.id)

    assert model.label == "测试test_edit_shield_resource_2"
    
    shield["label"]=""
    resource.shield.edit_shield(shield)
    model = Shield.objects.get(id=model.id)

    assert model.label == ""


test_edit_shield_resource()



测试test_edit_shield_resource 360
测试test_edit_shield_resource_2


In [6]:
# 测试屏蔽详情
def test_shield_detail():
    label = "测试test_shield_detail"
    model = create_shield(label)

    result = resource.shield.shield_detail({"id": model.id, "bk_biz_id": model.bk_biz_id})
    assert result["label"] == label


test_shield_detail()

In [7]:
# 测试前端屏蔽列表和后端屏蔽列表
def test_shield_list_and_frontend_shield_list():
    Shield.objects.all().delete()

    label_list = [
        "西游记",
        "东游记",
        "三国演义",
        "封神演义",
        "水浒传",
        "红楼梦",
        "封神榜",
        "搜神记",
        "大话西游",
    ]

    for lable in label_list:
        new_shield["label"] = lable
        resource.shield.add_shield(new_shield)

    # 精确查询测试
    params = {
        "page": 1,
        "page_size": 10,
        "time_range": "",
        "categories": [
            "alert",
            "scope",
            "strategy",
            "dimension"
        ],
        "search": "",
        "conditions": [
            {
                "key": "label",
                "value": "大话西游"
            },
        ],
        "is_active": True,
        "bk_biz_id": 2
    }

    # 测试后端精确查询
    result = resource.shield.shield_list(**params)
    assert result["shield_list"][0]["label"] == "大话西游"

    # 测试前端模糊查询
    params["conditions"] = []
    params["search"] = "游记"
    result = resource.shield.frontend_shield_list(**params)

    expect_value = ["西游记", "东游记"]
    expect_value.sort()
    actual_value = [item["label"] for item in result["shield_list"]]
    actual_value.sort()
    assert expect_value == actual_value


test_shield_list_and_frontend_shield_list()

In [8]:
# 测试前端屏蔽详情
def test_frontend_shield_detail():
    label = "测试test_frontend_shield_detail"
    model = create_shield(label)
    result = resource.shield.frontend_shield_detail({"id": model.id, "bk_biz_id": model.bk_biz_id})
    assert result["label"] == label


test_frontend_shield_detail()


In [9]:
# 测试前端克隆屏蔽
def test_fronend_clone_info():
    label = "测试test_fronend_clone_info"
    model = create_shield(label)
    result = resource.shield.frontend_clone_info({"id": model.id, "bk_biz_id": model.bk_biz_id})
    assert result["label"] == label


test_fronend_clone_info()